# Face Detection using TAO FaceNet

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. 

Train Adapt Optimize (TAO) Toolkit is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

<img align="center" src="https://developer.nvidia.com/sites/default/files/akamai/embedded-transfer-learning-toolkit-software-stack-1200x670px.png" width="1080"> 

## Learning Objectives
In this notebook, you will learn how to leverage the simplicity and convenience of TAO to:

* Take a pretrained resnet18 model and train a ResNet-18 FaceNet model on the WIDERFACE dataset
* Prune the trained FaceNet model
* Retrain the pruned model to recover lost accuracy
* Export the pruned model
* Run Inference on the trained model
* Export the pruned and retrained model to a .etlt file for deployment to DeepStream SDK
* Run inference on the exported. etlt model to verify deployment using TensorRT


### Table of Contents

This notebook shows an example use case of Face Detection using FaceNet in the Train Adapt Optimize (TAO) Toolkit.

0. [Set up env variables, map drives, and install dependencies](#head-0)
1. [Install the TAO Launcher](#head-1)
2. [Prepare dataset and pre-trained model](#head-2)
    1. [Verify and prepare dataset](#head-2-1)
    2. [Prepare tfrecords from kitti format dataset](#head-2-2)
    3. [Download pre-trained model](#head-2-3)
3. [Provide training specification](#head-3)
4. [Run TAO training](#head-4)
5. [Evaluate trained models](#head-5)
6. [Prune trained models](#head-6)
7. [Retrain pruned models](#head-7)
8. [Evaluate retrained model](#head-8)
9. [Visualize inferences](#head-9)
10. [Deploy](#head-10)
    1. [Int8 Optimization](#head-10-1)
    2. [Generate TensorRT engine](#head-10-2)
11. [Verify Deployed Model](#head-11)
    1. [Inference using TensorRT engine](#head-11-1)
    2. [Evaluate TensorRT engine](#head-11-2)

## 0. Set up env variables, map drives and install dependencies <a class="anchor" id="head-0"></a>

When using the purpose-built pretrained models from NGC, please make sure to set the `$KEY` environment variable to the key as mentioned in the model overview. Failing to do so, can lead to errors when trying to load them as pretrained models.

*Note: Please make sure to remove any stray artifacts/files from the `$LOCAL_PROJECT_DIR` paths as mentioned below, that may have been generated from previous experiments. Having checkpoint files etc may interfere with creating a training graph for a new experiment.*

The following notebook requires the user to set an env variable called the `$LOCAL_PROJECT_DIR` as the path to the users' workspace. Please note that the dataset to run this notebook is expected to reside in the `$LOCAL_PROJECT_DIR/facenet/data`, while the TAO experiment generated collaterals will be output to `$LOCAL_PROJECT_DIR/facenet`. More information on how to set up the dataset and the supported steps in the TAO workflow are provided in the subsequent cells.

*Note: This notebook currently is by default set up to run training using 1 GPU. To use more GPU's please update the env variable `$NUM_GPUS` accordingly*

In [ ]:
# Setting up env variables for cleaner command-line commands.
import os

%env KEY=nvidia_tlt
%env NUM_GPUS=1
%env USER_EXPERIMENT_DIR=/workspace/tao-experiments/facenet
%env DATA_DOWNLOAD_DIR=/workspace/tao-experiments/facenet/data
%env SPECS_DIR=/workspace/tao-experiments/facenet/specs

# Set this path if you don't run the notebook from the samples directory.
# %env NOTEBOOK_ROOT=~/tao-samples/facenet

# Please define this local project directory that needs to be mapped to the TAO docker session.
# The dataset is expected to be present in $LOCAL_PROJECT_DIR/facenet/data, while the results for the steps
# in this notebook will be stored at $LOCAL_PROJECT_DIR/facenet
# !PLEASE MAKE SURE TO UPDATE THIS PATH!.
%env LOCAL_PROJECT_DIR=/path/to/local/tlt-experiments

# $PROJECT_DIR is the path to the sample notebook folder and the dependency folder
# $PROJECT_DIR/deps should exist for dependency installation
%env PROJECT_DIR=/path/to/local/ngc-collaterals/cv/samples/

os.environ["LOCAL_DATA_DIR"] = os.path.join(
    os.getenv("LOCAL_PROJECT_DIR", os.getcwd()),
    "facenet/data"
)
os.environ["LOCAL_EXPERIMENT_DIR"] = os.path.join(
    os.getenv("LOCAL_PROJECT_DIR", os.getcwd()),
    "facenet"
)

# The sample spec files are present in the same path as the downloaded samples.
os.environ["LOCAL_SPECS_DIR"] = os.path.join(
    os.getenv("NOTEBOOK_ROOT", os.getcwd()),
    "specs"
)

# Showing list of specification files.
!ls -rlt $LOCAL_SPECS_DIR
!ls -rlt $LOCAL_DATASET_SPECS_DIR

The cell below maps the project directory on your local host to a workspace directory in the TAO docker instance, so that the data and the results are mapped from in and out of the docker. For more information please refer to the [launcher instance](https://docs.nvidia.com/tao/tao-toolkit/tao_launcher.html) in the user guide.

When running this cell on AWS, update the drive_map entry with the dictionary defined below, so that you don't have permission issues when writing data into folders created by the TAO docker.

```json
drive_map = {
    "Mounts": [
            # Mapping the data directory
            {
                "source": os.environ["LOCAL_PROJECT_DIR"],
                "destination": "/workspace/tao-experiments"
            },
            # Mapping the specs directory.
            {
                "source": os.environ["LOCAL_SPECS_DIR"],
                "destination": os.environ["SPECS_DIR"]
            },
        ],
    "DockerOptions": {
        "user": "{}:{}".format(os.getuid(), os.getgid())
    }
}
```

In [ ]:
# Mapping up the local directories to the TAO docker.
import json
mounts_file = os.path.expanduser("~/.tao_mounts.json")

# Define the dictionary with the mapped drives
drive_map = {
    "Mounts": [
        # Mapping the data directory
        {
            "source": os.environ["LOCAL_PROJECT_DIR"],
            "destination": "/workspace/tao-experiments"
        },
        # Mapping the specs directory.
        {
            "source": os.environ["LOCAL_SPECS_DIR"],
            "destination": os.environ["SPECS_DIR"]
        },
    ]
}

# Writing the mounts file.
with open(mounts_file, "w") as mfile:
    json.dump(drive_map, mfile, indent=4)

In [ ]:
!cat ~/.tao_mounts.json

In [ ]:
# Install requirement
!pip3 install -r $PROJECT_DIR/deps/requirements-pip.txt

## 1. Install the TAO launcher <a class="anchor" id="head-1"></a>
The TAO launcher is a python package distributed as a python wheel listed in the `nvidia-pyindex` python index. You may install the launcher by executing the following cell.

Please note that TAO Toolkit recommends users run the TAO launcher in a virtual env with python 3.6.9. You may follow the instruction on this [page](https://virtualenvwrapper.readthedocs.io/en/latest/install.html) to set up a python virtual env using the `virtualenv` and `virtualenvwrapper` packages. Once you have set up virtualenvwrapper, please set the version of python to be used in the virtual env by using the `VIRTUALENVWRAPPER_PYTHON` variable. You may do so by running

```sh
export VIRTUALENVWRAPPER_PYTHON=/path/to/bin/python3.x
```
where x >= 6 and <= 8

We recommend performing this step first and then launching the notebook from the virtual environment. In addition to installing TAO python package, please make sure of the following software requirements:
* python >=3.6.9 < 3.8.x
* docker-ce > 19.03.5
* docker-API 1.40
* nvidia-container-toolkit > 1.3.0-1
* nvidia-container-runtime > 3.4.0-1
* nvidia-docker2 > 2.5.0-1
* nvidia-driver > 455+

Once you have installed the pre-requisites, please log in to the docker registry nvcr.io by following the command below

```sh
docker login nvcr.io
```

You will be triggered to enter a username and password. The username is `$oauthtoken` and the password is the API key generated from `ngc.nvidia.com`. Please follow the instructions in the [NGC setup guide](https://docs.nvidia.com/ngc/ngc-overview/index.html#generating-api-key) to generate your own API key.

In [ ]:
# SKIP this step IF you have already installed the TAO launcher wheel.
!pip3 install nvidia-pyindex
!pip3 install nvidia-tao

In [ ]:
# View the versions of the TAO launcher
!tao info

## 2. Prepare dataset and pre-trained model <a class="anchor" id="head-2"></a>

 We will be using the Wider Face dataset for the tutorial. To find more details, please visit http://shuoyang1213.me/WIDERFACE/. Please download the [training](https://drive.google.com/file/d/0B6eKvaijfFUDQUUwd21EckhUbWs/view?usp=sharing) and [validation](https://drive.google.com/file/d/0B6eKvaijfFUDd3dIRmpvSk8tLUk/view?usp=sharing) set images, and the ground truth [labels](http://mmlab.ie.cuhk.edu.hk/projects/WIDERFace/support/bbx_annotation/wider_face_split.zip), and place the zip files in `$DATA_DOWNLOAD_DIR`. 

Notes: 
1. The detection workflow requires dataset to be in kitti format and we will need to convert it.
2. The pretrained facenet is trained on grayscale images of height 416 pixels and width 736 pixels. We will convert wider dataset to this format.


### 2.1. Verify and prepare dataset <a class="anchor" id="head-2-1"></a>

In [ ]:
# Check the dataset is present
!mkdir -p $LOCAL_DATA_DIR
!if [ ! -f $LOCAL_DATA_DIR/WIDER_train.zip ]; then echo 'Train Image zip file not found, please download.'; else echo 'Found Train Image zip file.';fi
!if [ ! -f $LOCAL_DATA_DIR/WIDER_val.zip ]; then echo 'Validation Image zip file not found, please download.'; else echo 'Found Validation Image zip file.';fi
!if [ ! -f $LOCAL_DATA_DIR/wider_face_split.zip ]; then echo 'Label zip file not found, please download.'; else echo 'Found Labels zip file.';fi

In [ ]:
# unpack downloaded datasets to $DATA_DOWNLOAD_DIR.
!unzip -u $LOCAL_DATA_DIR/WIDER_train.zip -d $LOCAL_DATA_DIR
!unzip -u $LOCAL_DATA_DIR/WIDER_val.zip -d $LOCAL_DATA_DIR
!unzip -u $LOCAL_DATA_DIR/wider_face_split.zip -d $LOCAL_DATA_DIR

In [ ]:
# verify
!ls -l $LOCAL_DATA_DIR/WIDER_train/
!ls -l $LOCAL_DATA_DIR/WIDER_val/
!ls -l $LOCAL_DATA_DIR/wider_face_split/

In [ ]:
# Convert wider train dataset to kitti format
!python3.6 convert_wider_to_kitti.py --input_image_dir=$LOCAL_DATA_DIR/WIDER_train/images \
                                     --input_label_file=$LOCAL_DATA_DIR/wider_face_split/wider_face_train_bbx_gt.txt \
                                     --output_dir=$LOCAL_DATA_DIR/training/ \
                                     --image_height=416 --image_width=736 --grayscale

In [ ]:
# Convert wider validation dataset to kitti format
!python3.6 convert_wider_to_kitti.py --input_image_dir=$LOCAL_DATA_DIR/WIDER_val/images \
                                     --input_label_file=$LOCAL_DATA_DIR/wider_face_split/wider_face_val_bbx_gt.txt \
                                     --output_dir=$LOCAL_DATA_DIR/validation/ \
                                     --image_height=416 --image_width=736 --grayscale

In [ ]:
# verify
import os

DATA_DIR = os.environ.get('LOCAL_DATA_DIR')
num_training_images = len(os.listdir(os.path.join(DATA_DIR, "training/images")))
num_training_labels = len(os.listdir(os.path.join(DATA_DIR, "training/labels")))
num_val_images = len(os.listdir(os.path.join(DATA_DIR, "validation/images")))
num_val_labels = len(os.listdir(os.path.join(DATA_DIR, "validation/labels")))
print("Number of images in the training set. {}".format(num_training_images))
print("Number of labels in the training set. {}".format(num_training_labels))
print("Number of images in the validation set. {}".format(num_val_images))
print("Number of labels in the validation set. {}".format(num_val_labels))

In [ ]:
# Sample kitti label.
!cat $LOCAL_DATA_DIR/training/labels/30_Surgeons_Surgeons_30_227.txt

### 2.2. Prepare tf records from kitti format dataset <a class="anchor" id="head-2-2"></a>

* Update the tfrecords spec file to take in your kitti format dataset
* Create the tfrecords using the detectnet_v2 dataset_convert 

*Note: TfRecords only need to be generated once.*

In [ ]:
print("TFrecords conversion spec file for kitti training")
!cat $LOCAL_SPECS_DIR/facenet_tfrecords_kitti_train.txt

In [ ]:
# Creating a new directory for the output tfrecords dump.
print("Converting Tfrecords for wider train dataset")
!mkdir -p $LOCAL_DATA_DIR/tfrecords && rm -rf $LOCAL_DATA_DIR/tfrecords/*
!tao detectnet_v2 dataset_convert \
                  -d $SPECS_DIR/facenet_tfrecords_kitti_train.txt \
                  -o $DATA_DOWNLOAD_DIR/tfrecords/training/kitti_train

In [ ]:
# Creating a new directory for the output tfrecords dump.
print("Converting Tfrecords for wider validation dataset")
!tao detectnet_v2 dataset_convert \
                  -d $SPECS_DIR/facenet_tfrecords_kitti_val.txt \
                  -o $DATA_DOWNLOAD_DIR/tfrecords/validation/kitti_val

In [ ]:
!ls -rlt $LOCAL_DATA_DIR/tfrecords/training/

In [ ]:
!ls -rlt $LOCAL_DATA_DIR/tfrecords/validation/

### 2.3. Download pre-trained model <a class="anchor" id="head-2-3"></a>

Download the correct pretrained model from the NGC model registry for your experiment. Please note that for DetectNet_v2, the input is expected to be 0-1 normalized.

For FaceNet pretrain model please download from: `nvidia/tao/facenet`

After downloading the pre-trained model, please place the files in $LOCAL_EXPERIMENT_DIR
You will then have the following path

* pretrain model in `$LOCAL_EXPERIMENT_DIR/pretrain_models/facenet_vunpruned_v2.0/model.tlt`

In [ ]:
# Installing NGC CLI on the local machine.
## Download and install
%env CLI=ngccli_cat_linux.zip
!mkdir -p $LOCAL_PROJECT_DIR/ngccli

# Remove any previously existing CLI installations
!rm -rf $LOCAL_PROJECT_DIR/ngccli/*
!wget "https://ngc.nvidia.com/downloads/$CLI" -P $LOCAL_PROJECT_DIR/ngccli
!unzip -u "$LOCAL_PROJECT_DIR/ngccli/$CLI" -d $LOCAL_PROJECT_DIR/ngccli/
!rm $LOCAL_PROJECT_DIR/ngccli/*.zip 
os.environ["PATH"]="{}/ngccli:{}".format(os.getenv("LOCAL_PROJECT_DIR", ""), os.getenv("PATH", ""))

In [ ]:
# List models available in the model registry.
!ngc registry model list nvidia/tao/facenet:unpruned*

In [ ]:
# Position the pretrain model to the target destination.
!mkdir -p $LOCAL_EXPERIMENT_DIR/pretrain_models

In [ ]:
# Download the pretrained model from NGC
!ngc registry model download-version nvidia/tao/facenet:unpruned_v2.0 \
    --dest $LOCAL_EXPERIMENT_DIR/pretrain_models

In [ ]:
# Check the pretrained model is present
!if [ ! -f $LOCAL_EXPERIMENT_DIR/pretrain_models/facenet_vunpruned_v2.0/model.tlt ]; then echo 'Pretrain model file not found, please download.'; else echo 'Found Pretrain model file.';fi

## 3. Provide training specification <a class="anchor" id="head-3"></a>
* Tfrecords for the train datasets: In order to use the newly generated tfrecords, update the dataset_config parameter in the spec file at `$LOCAL_SPECS_DIR/facenet_train_resnet18_kitti.txt` 
* Augmentation parameters for on the fly data augmentation
* Other training (hyper-)parameters such as batch size, number of epochs, learning rate etc.

*Note: Please set the `load_graph` option to `true` in the model_config to load the pretrained facenet model.*

In [ ]:
!cat $LOCAL_SPECS_DIR/facenet_train_resnet18_kitti.txt

## 4. Run TAO training <a class="anchor" id="head-4"></a>
* Provide the sample spec file and the output directory location for models

*Note: The training may take hours to complete. Also, the remaining notebook, assumes that the training was done in single-GPU mode. When run in multi-GPU mode, please expect to update the pruning and inference steps with new pruning thresholds and updated parameters in the clusterfile.json accordingly for optimum performance.*

*Detectnet_v2 now supports restart from checkpoint. In case, the training job is killed prematurely, you may resume training from the closest checkpoint by simply re-running the **same** command line. Please do make sure to use the <u>**same number of GPUs**</u> when restarting the training.*

*When running the training with NUM_GPUs>1, you may need to modify the `batch_size_per_gpu` and `learning_rate` to get similar mAP as a 1GPU training run. In most cases, scaling down the batch-size by a factor of NUM_GPU's or scaling up the learning rate by a factor of NUM_GPU's would be a good place to start.* 

First, we evaluate the pretrained Face Detect network on Wider validation set

In [ ]:
!tao detectnet_v2 evaluate -e $SPECS_DIR/facenet_train_resnet18_kitti.txt\
                           -m $USER_EXPERIMENT_DIR/pretrain_models/facenet_vunpruned_v2.0/model.tlt\
                           -k $KEY

Next, we train model on Wider train set starting with the pretrained Face Detect model weights

In [ ]:
!tao detectnet_v2 train -e $SPECS_DIR/facenet_train_resnet18_kitti.txt \
                        -r $USER_EXPERIMENT_DIR/experiment_dir_unpruned \
                        -k $KEY \
                        -n resnet18_detector \
                        --gpus $NUM_GPUS

In [ ]:
print('Model for each epoch:')
print('---------------------')
!ls -lh $LOCAL_EXPERIMENT_DIR/

## 5. Evaluate the trained model <a class="anchor" id="head-5"></a>

In [ ]:
!tao detectnet_v2 evaluate -e $SPECS_DIR/facenet_train_resnet18_kitti.txt\
                           -m $USER_EXPERIMENT_DIR/experiment_dir_unpruned/weights/resnet18_detector.tlt \
                           -k $KEY

## 6. Prune the trained model <a class="anchor" id="head-6"></a>
* Specify pre-trained model
* Equalization criterion
* Threshold for pruning.
* A key to save and load the model
* Output directory to store the model

*Usually, you just need to adjust `-pth` (threshold) for accuracy and model size trade off. Higher `pth` gives you smaller model (and thus higher inference speed) but worse accuracy. The threshold to use depends on the dataset. A pth value `5.2e-6` is just a start point. If the retrain accuracy is good, you can increase this value to get smaller models. Otherwise, lower this value to get better accuracy.*

*For some internal studies, we have noticed that a pth value of 0.01 is a good starting point for detectnet_v2 models.*

In [ ]:
# Create an output directory if it doesn't exist.
!mkdir -p $LOCAL_EXPERIMENT_DIR/experiment_dir_pruned

In [ ]:
!tao detectnet_v2 prune \
                  -m $USER_EXPERIMENT_DIR/experiment_dir_unpruned/weights/resnet18_detector.tlt \
                  -o $USER_EXPERIMENT_DIR/experiment_dir_pruned/resnet18_nopool_bn_detectnet_v2_pruned.tlt \
                  -eq union \
                  -pth 0.0000052 \
                  -k $KEY

In [ ]:
!ls -rlt $LOCAL_EXPERIMENT_DIR/experiment_dir_pruned/

## 7. Retrain the pruned model <a class="anchor" id="head-7"></a>
* Model needs to be re-trained to bring back accuracy after pruning
* Specify re-training specification with pretrained weights as pruned model.

*Note: For retraining, please set the `load_graph` option to `true` in the model_config to load the pruned model graph. Also, if after retraining, the model shows some decrease in mAP, it could be that the originally trained model, was pruned a little too much. Please try reducing the pruning threshold, thereby reducing the pruning ratio, and use the new model to retrain.*

In [ ]:
# Printing the retrain experiment file. 
# Note: We have updated the experiment file to include the 
# newly pruned model as a pretrained weights and, the
# load_graph option is set to true 
!cat $LOCAL_SPECS_DIR/facenet_retrain_resnet18_kitti.txt

In [ ]:
# Retraining using the pruned model as pretrained weights 
!tao detectnet_v2 train -e $SPECS_DIR/facenet_retrain_resnet18_kitti.txt \
                        -r $USER_EXPERIMENT_DIR/experiment_dir_retrain \
                        -k $KEY \
                        -n resnet18_detector_pruned \
                        --gpus $NUM_GPUS

In [ ]:
# Listing the newly retrained model.
!ls -rlt $LOCAL_EXPERIMENT_DIR/experiment_dir_retrain/weights

## 8. Evaluate the retrained model <a class="anchor" id="head-8"></a>

This section evaluates the pruned and retrained model, using `tao-evaluate`.

In [ ]:
!tao detectnet_v2 evaluate -e $SPECS_DIR/facenet_retrain_resnet18_kitti.txt \
                           -m $USER_EXPERIMENT_DIR/experiment_dir_retrain/weights/resnet18_detector_pruned.tlt \
                           -k $KEY

## 9. Visualize inferences <a class="anchor" id="head-9"></a>
In this section, we run the `inference` tool to generate inferences on the trained models.

In [ ]:
# Running inference for detection on n images
!tao detectnet_v2 inference -e $SPECS_DIR/facenet_inference_kitti_tlt.txt \
                            -o $USER_EXPERIMENT_DIR/tlt_infer_testing \
                            -i $DATA_DOWNLOAD_DIR/validation/images \
                            -k $KEY

The `infer` tool produces two outputs. 
1. Overlain images in `$USER_EXPERIMENT_DIR/tlt_infer_testing/images_annotated`
2. Frame by frame bbox labels in kitti format located in `$USER_EXPERIMENT_DIR/tlt_infer_testing/labels`

*Note: To run inferences for a single image, simply replace the path to the -i flag in `infer` command with the path to the image.*

In [ ]:
# Simple grid visualizer
%matplotlib inline
import matplotlib.pyplot as plt
import os
from math import ceil
valid_image_ext = ['.jpg', '.png', '.jpeg', '.ppm']

def visualize_images(image_dir, num_cols=4, num_images=10):
    output_path = os.path.join(os.environ['LOCAL_EXPERIMENT_DIR'], image_dir)
    num_rows = int(ceil(float(num_images) / float(num_cols)))
    f, axarr = plt.subplots(num_rows, num_cols, figsize=[80,30])
    f.tight_layout()
    a = [os.path.join(output_path, image) for image in os.listdir(output_path) 
         if os.path.splitext(image)[1].lower() in valid_image_ext]
    for idx, img_path in enumerate(a[:num_images]):
        col_id = idx % num_cols
        row_id = idx // num_cols
        img = plt.imread(img_path)
        axarr[row_id, col_id].imshow(img) 

In [ ]:
# Visualizing the first 12 images.
OUTPUT_PATH = 'tlt_infer_testing/images_annotated' # relative path from $USER_EXPERIMENT_DIR.
COLS = 4 # number of columns in the visualizer grid.
IMAGES = 12 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)

## 10. Deploy! <a class="anchor" id="head-10"></a>

In [ ]:
!mkdir -p $LOCAL_EXPERIMENT_DIR/experiment_dir_final
# Removing a pre-existing copy of the etlt if there has been any.
import os
output_file=os.path.join(os.environ['LOCAL_EXPERIMENT_DIR'],
                         "experiment_dir_final/resnet18_detector.etlt")
if os.path.exists(output_file):
    os.system("rm {}".format(output_file))
!tao detectnet_v2 export \
            -m $USER_EXPERIMENT_DIR/experiment_dir_retrain/weights/resnet18_detector_pruned.tlt \
            -o $USER_EXPERIMENT_DIR/experiment_dir_final/resnet18_detector.etlt \
            -k $KEY

In [ ]:
print('Exported model:')
print('------------')
!ls -lh $LOCAL_EXPERIMENT_DIR/experiment_dir_final

### 10.1. Int8 Optimization <a class="anchor" id="head-10-1"></a>
DetectNet_v2 model supports int8 inference mode in TensorRT. 
In order to use int8 mode, we must calibrate the model to run 8-bit inferences -

* Generate calibration tensorfile from the training data using detectnet_v2 calibration_tensorfile
* Use tao <task> export to generate int8 calibration table.

*Note: For this example, we generate a calibration tensorfile containing 10 batches of training data.
Ideally, it is best to use at least 10-20% of the training data to do so. The more data provided during calibration, the closer int8 inferences are to fp32 inferences.*

In [ ]:
!tao detectnet_v2 calibration_tensorfile -e $SPECS_DIR/facenet_retrain_resnet18_kitti.txt \
                                         -m 10 \
                                         -o $USER_EXPERIMENT_DIR/experiment_dir_final/calibration.tensor

In [ ]:
!rm -rf $LOCAL_EXPERIMENT_DIR/experiment_dir_final/resnet18_detector.etlt
!rm -rf $LOCAL_EXPERIMENT_DIR/experiment_dir_final/calibration.bin
!tao detectnet_v2 export \
                  -m $USER_EXPERIMENT_DIR/experiment_dir_retrain/weights/resnet18_detector_pruned.tlt \
                  -o $USER_EXPERIMENT_DIR/experiment_dir_final/resnet18_detector.etlt \
                  -k $KEY  \
                  --cal_data_file $USER_EXPERIMENT_DIR/experiment_dir_final/calibration.tensor \
                  --data_type int8 \
                  --batches 10 \
                  --batch_size 32 \
                  --max_batch_size 32 \
                  --engine_file $USER_EXPERIMENT_DIR/experiment_dir_final/resnet18_detector.trt.int8 \
                  --cal_cache_file $USER_EXPERIMENT_DIR/experiment_dir_final/calibration.bin \
                  --verbose

### 10.2. Generate TensorRT engine <a class="anchor" id="head-10-2"></a>
Verify engine generation using the `tao-converter` utility included with the docker.

The `tao-converter` produces optimized tensorrt engines for the platform that it resides on. Therefore, to get maximum performance, please instantiate this docker and execute the `tao-converter` command, with the exported `.etlt` file on your target device. The tao-converter utility included in this docker only works for x86 devices, with discrete NVIDIA GPU's. 

For the jetson devices, please download the tao-converter for jetson from the dev zone link [here](https://developer.nvidia.com/tao-converter). 

In [ ]:
!tao converter $USER_EXPERIMENT_DIR/experiment_dir_final/resnet18_detector.etlt \
               -k $KEY \
               -o output_cov/Sigmoid,output_bbox/BiasAdd \
               -d 3,416,736 \
               -i nchw \
               -m 64 \
               -t fp16 \
               -e $USER_EXPERIMENT_DIR/experiment_dir_final/resnet18_detector.trt \
               -b 32

## 11. Verify Deployed Model <a class="anchor" id="head-11"></a>

### 11.1. Inference using TensorRT engine <a class="anchor" id="head-11-1"></a>

Verify the exported model by visualizing inferences on TensorRT.
In addition to running inference on a `.tlt` model in [step 9](#head-9), the `inference` tool is also capable of consuming the converted `TensorRT engine` from [step 10](#head-10).

In [ ]:
!tao detectnet_v2 inference -e $SPECS_DIR/facenet_inference_kitti_etlt.txt \
                            -o $USER_EXPERIMENT_DIR/etlt_infer_testing \
                            -i $DATA_DOWNLOAD_DIR/validation/images \
                            -k $KEY

In [ ]:
# visualize the first 12 inferenced images.
OUTPUT_PATH = 'etlt_infer_testing/images_annotated' # relative path from $USER_EXPERIMENT_DIR.
COLS = 4 # number of columns in the visualizer grid.
IMAGES = 12 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)

### 11.2. Evaluate using TensorRT engine <a class="anchor" id="head-11-2"></a>

Verify the exported model by evaluating the TensorRT.
In addition to running inference on a `.tlt` model in [step 9](#head-9), the `inference` tool is also capable of consuming the converted `TensorRT engine` from [step 10](#head-10).

In [ ]:
!tao detectnet_v2 evaluate -e $SPECS_DIR/facenet_retrain_resnet18_kitti.txt \
                           -m $USER_EXPERIMENT_DIR/experiment_dir_final/resnet18_detector.trt.int8 \
                           -k $KEY \
                           -f tensorrt